In [1]:
import pandas as pd
import numpy as np 
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F
from torchsummary import summary
from torch.autograd import Variable

# Load data

In [2]:
df = pd.read_csv(r'..\Dataset\SWAT\SWaT.csv') 
df.insert(78,'Label','Normal')
#Attack on FIT401
df.loc[((df['Timestamp'] >='2019-07-20T07:08:46Z') & (df['Timestamp'] < '2019-07-20T07:10:311Z')),'Label']='Attack'
#Attack on LIT301
df.loc[((df['Timestamp'] >='2019-07-20T07:15:00Z') & (df['Timestamp'] < '2019-07-20T07:19:321Z')),'Label']='Attack'
#Attack on P601
df.loc[((df['Timestamp'] >='2019-07-20T07:26:57Z') & (df['Timestamp'] < '2019-07-20T07:30:481Z')),'Label']='Attack'
#Multipoint Attack
df.loc[((df['Timestamp'] >='2019-07-20T07:38:50Z') & (df['Timestamp'] < '2019-07-20T07:46:201Z')),'Label']='Attack'
##Attack on MV501
df.loc[((df['Timestamp'] >='2019-07-20T07:54:00Z') & (df['Timestamp'] < '2019-07-20T07:56:01Z')),'Label']='Attack'
#Attack on P301
df.loc[((df['Timestamp'] >='2019-07-20T08:02:56Z') & (df['Timestamp'] < '2019-07-20T08:16:181Z')),'Label']='Attack'


In [3]:
df

,Timestamp,FIT 101,LIT 101,MV 101,P1_STATE,P101 Status,P102 Status,AIT 201,AIT 202,AIT 203,...,LSH 602,LSH 603,LSL 601,LSL 602,LSL 603,P6 STATE,P601 Status,P602 Status,P603 Status,Label
0,2019-07-20T04:30:00Z,0.000000,729.865800,1,3,2,1,142.527557,9.293002,198.077423,...,Active,Inactive,Inactive,Inactive,Active,2,1,1,1,Normal
1,2019-07-20T04:30:01Z,0.000000,729.434000,1,3,2,1,142.527557,9.293002,198.385025,...,Active,Inactive,Inactive,Inactive,Active,2,1,1,1,Normal
2,2019-07-20T04:30:02.004013Z,0.000000,729.120000,1,3,2,1,142.527557,9.293002,198.436300,...,Active,Inactive,Inactive,Inactive,Active,2,1,1,1,Normal
3,2019-07-20T04:30:03.004013Z,0.000000,728.688200,1,3,2,1,142.527557,9.289157,198.667000,...,Active,Inactive,Inactive,Inactive,Active,2,1,1,1,Normal
4,2019-07-20T04:30:04Z,0.000000,727.706900,1,3,2,1,142.527557,9.289157,198.897720,...,Active,Inactive,Inactive,Inactive,Active,2,1,1,1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14991,2019-07-20T08:39:55.001007Z,4.200429,491.169769,2,2,2,1,131.408615,9.319918,257.703156,...,Active,Inactive,Inactive,Inactive,Active,2,1,1,1,Normal
14992,2019-07-20T08:39:56.0050048Z,4.253915,491.405273,2,2,2,1,131.408615,9.317354,257.703156,...,Active,Inactive,Inactive,Inactive,Active,2,1,1,1,Normal
14993,2019-07-20T08:39:57.0050048Z,4.303558,492.308100,2,2,2,1,131.408615,9.317354,257.703156,...,Active,Inactive,Inactive,Inactive,Active,2,1,1,1,Normal
14994,2019-07-20T08:39:58.0050048Z,4.323736,492.465100,2,2,2,1,131.408615,9.316713,257.703156,...,Active,Inactive,Inactive,Inactive,Active,2,1,1,1,Normal


In [4]:
df.describe()

,FIT 101,LIT 101,MV 101,P1_STATE,P101 Status,P102 Status,AIT 201,AIT 202,AIT 203,FIT 201,...,P501 Status,P502 Status,PIT 501,PIT 502,PIT 503,FIT 601,P6 STATE,P601 Status,P602 Status,P603 Status
count,14996.000000,14996.000000,14996.000000,14996.000000,14996.000000,14996.0,14996.000000,14996.000000,14996.000000,14996.000000,...,14996.0,14996.0,14996.000000,14996.000000,14996.000000,14996.000000,14996.0,14996.000000,14996.0,14996.0
mean,0.740876,733.960251,1.156175,2.061616,1.374166,1.0,138.766501,9.210022,247.985162,0.869760,...,2.0,1.0,160.706744,4.673115,115.157048,0.000402,2.0,1.015337,1.0,1.0
std,1.634632,110.960185,0.384272,0.240466,0.483923,0.0,8.265845,0.175812,11.806186,1.121283,...,0.0,0.0,5.707502,18.183883,5.324942,0.003317,0.0,0.122895,0.0,0.0
min,0.000000,491.169769,0.000000,2.000000,1.000000,1.0,113.849014,8.768457,198.077423,0.000000,...,2.0,1.0,158.132523,2.450902,111.654060,0.000000,2.0,1.000000,1.0,1.0
25%,0.000000,640.595184,1.000000,2.000000,1.000000,1.0,131.536789,9.090170,239.887200,0.000384,...,2.0,1.0,159.526400,2.851376,114.233528,0.000256,2.0,1.000000,1.0,1.0
50%,0.000000,819.636841,1.000000,2.000000,1.000000,1.0,143.713150,9.233082,246.218918,0.000513,...,2.0,1.0,160.231354,2.883414,114.714172,0.000320,2.0,1.000000,1.0,1.0
75%,0.000000,820.971436,1.000000,2.000000,2.000000,1.0,144.033585,9.345873,257.190460,2.320187,...,2.0,1.0,160.695969,2.963509,115.002563,0.000320,2.0,1.000000,1.0,1.0
max,4.403484,825.092957,2.000000,3.000000,2.000000,1.0,146.821335,9.490067,272.289154,2.342357,...,2.0,1.0,219.014359,192.371765,170.565247,0.137315,2.0,2.000000,1.0,1.0


In [5]:
df.columns

Index(['Timestamp', 'FIT 101', 'LIT 101', 'MV 101', 'P1_STATE', 'P101 Status',
       'P102 Status', 'AIT 201', 'AIT 202', 'AIT 203', 'FIT 201', 'LS 201',
       'LS 202', 'LSL 203', 'LSLL 203', 'MV201', 'P2_STATE', 'P201 Status',
       'P202 Status', 'P203 Status', 'P204 Status', 'P205 Status',
       'P206 Status', 'P207 Status', 'P208 Status', 'AIT 301', 'AIT 302',
       'AIT 303', 'DPIT 301', 'FIT 301', 'LIT 301', 'MV 301', 'MV 302',
       'MV 303', 'MV 304', 'P3_STATE', 'P301 Status', 'P302 Status', 'AIT 401',
       'AIT 402', 'FIT 401', 'LIT 401', 'LS 401', 'P4_STATE', 'P401 Status',
       'P402 Status', 'P403 Status', 'P404 Status', 'UV401', 'AIT 501',
       'AIT 502', 'AIT 503', 'AIT 504', 'FIT 501', 'FIT 502', 'FIT 503',
       'FIT 504', 'MV 501', 'MV 502', 'MV 503', 'MV 504', 'P5_STATE',
       'P501 Status', 'P502 Status', 'PIT 501', 'PIT 502', 'PIT 503',
       'FIT 601', 'LSH 601', 'LSH 602', 'LSH 603', 'LSL 601', 'LSL 602',
       'LSL 603', 'P6 STATE', 'P601 Statu

In [6]:
df['Label'].value_counts()

Normal    13016
Attack     1980
Name: Label, dtype: int64

In [7]:
test_df = df.drop(['Timestamp'],axis=1)

In [8]:
# Get list of categorical variables
s = (test_df.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['LS 201', 'LS 202', 'LSL 203', 'LSLL 203', 'LS 401', 'LSH 601', 'LSH 602', 'LSH 603', 'LSL 601', 'LSL 602', 'LSL 603', 'Label']


In [9]:
from sklearn.preprocessing import LabelEncoder

label_df = test_df.copy()

# inactive = 0 active = 1     label: normal = 1 , attack = 0
label_encoder = LabelEncoder()
for col in object_cols:
    label_df[col] = label_encoder.fit_transform(test_df[col])

label_df

,FIT 101,LIT 101,MV 101,P1_STATE,P101 Status,P102 Status,AIT 201,AIT 202,AIT 203,FIT 201,...,LSH 602,LSH 603,LSL 601,LSL 602,LSL 603,P6 STATE,P601 Status,P602 Status,P603 Status,Label
0,0.000000,729.865800,1,3,2,1,142.527557,9.293002,198.077423,2.335437,...,0,0,0,0,0,2,1,1,1,1
1,0.000000,729.434000,1,3,2,1,142.527557,9.293002,198.385025,2.335437,...,0,0,0,0,0,2,1,1,1,1
2,0.000000,729.120000,1,3,2,1,142.527557,9.293002,198.436300,2.335437,...,0,0,0,0,0,2,1,1,1,1
3,0.000000,728.688200,1,3,2,1,142.527557,9.289157,198.667000,2.335437,...,0,0,0,0,0,2,1,1,1,1
4,0.000000,727.706900,1,3,2,1,142.527557,9.289157,198.897720,2.335437,...,0,0,0,0,0,2,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14991,4.200429,491.169769,2,2,2,1,131.408615,9.319918,257.703156,2.316086,...,0,0,0,0,0,2,1,1,1,1
14992,4.253915,491.405273,2,2,2,1,131.408615,9.317354,257.703156,2.314292,...,0,0,0,0,0,2,1,1,1,1
14993,4.303558,492.308100,2,2,2,1,131.408615,9.317354,257.703156,2.313651,...,0,0,0,0,0,2,1,1,1,1
14994,4.323736,492.465100,2,2,2,1,131.408615,9.316713,257.703156,2.313651,...,0,0,0,0,0,2,1,1,1,1


In [10]:
Y = label_df.Label
X = label_df.drop(['Label'], axis=1)

In [11]:
values = X

In [12]:
Y.value_counts()

1    13016
0     1980
Name: Label, dtype: int64

In [13]:
#normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_normalized=scaler.fit_transform(values)
X_normalized

array([[0.        , 0.71482317, 0.5       , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.71353006, 0.5       , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.71258972, 0.5       , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.97730752, 0.00340896, 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.98188965, 0.00387913, 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.98188965, 0.00517218, 1.        , ..., 0.        , 0.        ,
        0.        ]])

In [14]:
from sklearn.model_selection import train_test_split
# # Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_normalized, Y, train_size=0.9, test_size=0.1,
                                                                random_state=0, shuffle=False)

In [15]:
X_train_full.shape

(13496, 77)

In [16]:
#function to convert to time domain dataset
def create_dataset(X, time_steps):
    Xs = []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
    return np.array(Xs)

In [17]:
# reshape to [samples, time_steps, n_features]

TIME_STEPS = 1
X_train = pd.DataFrame(X_train_full)
X_train = create_dataset(X_train, TIME_STEPS)
y_train = pd.DataFrame(y_train)
y_train = create_dataset(y_train, TIME_STEPS)
X_valid = pd.DataFrame(X_valid_full)
X_valid = create_dataset(X_valid, TIME_STEPS)
y_valid = pd.DataFrame(y_valid)
y_valid = create_dataset(y_valid, TIME_STEPS)
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

(13495, 1, 77)
(13495, 1, 1)
(1499, 1, 77)
(1499, 1, 1)


# Construct Model

In [18]:
class LSTM(nn.Module):
    def __init__(self,input_size, output_size, hidden_size, num_layers):
        super(LSTM,self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        #layer 1
        self.lstm = nn.LSTM(self.input_size, 
                            self.hidden_size,
                            self.num_layers,
                            batch_first=False,dropout = 0)

        self.dropout = nn.Dropout(p=0.3)
        #layer 2
        self.fc = nn.Linear(hidden_size,output_size)
      
        self.sigmoid = nn.Sigmoid()
     
    def forward(self, x):     
        per_out=[]
        lstm_out, self.hidden_cell = self.lstm(x)
        per_out.append(lstm_out)
        lstm_out = self.dropout(lstm_out)    
        out = self.fc(lstm_out)
        per_out.append(out)
        score = self.sigmoid(out)
        return score, per_out

In [19]:
def evaluate_accuracy(x,y,model):
    output,pre_out = model(x)
    output = torch.reshape(output,[-1,1])
    correct = (output.ge(0.5) == y).sum().item()
    n = y.shape[0]
    return correct/n

In [20]:
input_size = 77
output_size = 1
hidden_size = 20
num_layers =2

device = torch.device('cuda')
model = LSTM(input_size,output_size, hidden_size, num_layers).to(device)
loss_function = nn.BCELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
# optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

In [21]:
def getDataloader(X_train_full,y_train,X_valid_full,y_valid):
    TIME_STEPS = 1
    X_train = pd.DataFrame(X_train_full)
    X_train = create_dataset(X_train, TIME_STEPS)
    y_train = pd.DataFrame(y_train)
    y_train = create_dataset(y_train, TIME_STEPS)
    X_valid = pd.DataFrame(X_valid_full)
    X_valid = create_dataset(X_valid, TIME_STEPS)
    y_valid = pd.DataFrame(y_valid)
    y_valid = create_dataset(y_valid, TIME_STEPS)
# 将输入和输出封装进Data.TensorDataset()类对象
    x = torch.tensor(X_train[:,:,:]).float()
    x = torch.where(torch.isnan(x), torch.full_like(x, 0), x)
    y = y_train[:]
    y = np.array(y)
# y = torch.tensor(np.reshape(y,[-1,1]))
    y = torch.tensor(y)
    y = y.float()
    torch_dataset = Data.TensorDataset(x,y)
    train_dataloader = torch.utils.data.DataLoader(torch_dataset, batch_size=60, shuffle=False, num_workers=0)
    return X_train, y_train, X_valid, y_valid, train_dataloader

In [22]:
import torch.utils.data as Data
# 将输入和输出封装进Data.TensorDataset()类对象
x = torch.tensor(X_train[:,:,:]).float()
x = torch.where(torch.isnan(x), torch.full_like(x, 0), x)
    
y = y_train[:]
y = np.array(y)
# y = torch.tensor(np.reshape(y,[-1,1]))
y = torch.tensor(y)
y = y.float()
print(x.shape)
print(y.shape)

torch_dataset = Data.TensorDataset(x,y)

torch.Size([13495, 1, 77])
torch.Size([13495, 1, 1])


In [23]:
BATCH_SIZE = 60
train_dataloader = torch.utils.data.DataLoader(torch_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [24]:
train_dataloader.batch_size

60

In [25]:
batch_array = np.zeros(len(train_dataloader))
batch_index_array = np.zeros(len(train_dataloader))

# Training and Evaluation

In [26]:
import time

time_start=time.time()

epochs = 20

for epoch in range(epochs):
    model.train()
    for batch, (batch_x, batch_y) in enumerate(train_dataloader):   
        batch_x = batch_x.cuda()   
        output,pre_out = model(batch_x)
        output = torch.reshape(output,[-1,1])

        batch_y = np.array(batch_y)
        batch_y = torch.tensor(np.reshape(batch_y,[-1,1]))
        batch_y = batch_y.float()
        batch_y = batch_y.cuda()

        loss = loss_function(output,batch_y)
        acc = evaluate_accuracy(batch_x,batch_y,model)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_array[batch] = loss.item()
        batch_index_array[batch] = batch
        if epoch % 5 == 0:
            print("epoch:{} batch:{} loss:{} acc:{}".format(epoch,batch,loss.item(),acc))

time_end=time.time()
print('totally cost',time_end-time_start)

epoch:0 batch:0 loss:0.7012244462966919 acc:0.26666666666666666
epoch:0 batch:1 loss:0.6923128366470337 acc:0.5333333333333333
epoch:0 batch:2 loss:0.6826073527336121 acc:0.7833333333333333
epoch:0 batch:3 loss:0.6670244932174683 acc:0.9
epoch:0 batch:4 loss:0.6533950567245483 acc:0.9166666666666666
epoch:0 batch:5 loss:0.6438571214675903 acc:0.95
epoch:0 batch:6 loss:0.6308839321136475 acc:0.95
epoch:0 batch:7 loss:0.6154579520225525 acc:0.9666666666666667
epoch:0 batch:8 loss:0.6109666228294373 acc:0.9833333333333333
epoch:0 batch:9 loss:0.6095412373542786 acc:0.9833333333333333
epoch:0 batch:10 loss:0.5880549550056458 acc:0.9833333333333333
epoch:0 batch:11 loss:0.575346827507019 acc:0.9833333333333333
epoch:0 batch:12 loss:0.5619862675666809 acc:0.9833333333333333
epoch:0 batch:13 loss:0.5488240718841553 acc:1.0
epoch:0 batch:14 loss:0.5303394794464111 acc:1.0
epoch:0 batch:15 loss:0.5204190015792847 acc:1.0
epoch:0 batch:16 loss:0.5036876201629639 acc:1.0
epoch:0 batch:17 loss:0.4

epoch:0 batch:172 loss:0.03721851855516434 acc:1.0
epoch:0 batch:173 loss:0.03754488006234169 acc:1.0
epoch:0 batch:174 loss:0.04117433354258537 acc:1.0
epoch:0 batch:175 loss:0.03714853152632713 acc:1.0
epoch:0 batch:176 loss:0.40109872817993164 acc:0.9
epoch:0 batch:177 loss:3.490652561187744 acc:0.0
epoch:0 batch:178 loss:3.523879289627075 acc:0.0
epoch:0 batch:179 loss:3.366628646850586 acc:0.0
epoch:0 batch:180 loss:2.493889808654785 acc:0.26666666666666666
epoch:0 batch:181 loss:0.049656178802251816 acc:1.0
epoch:0 batch:182 loss:0.05254127457737923 acc:1.0
epoch:0 batch:183 loss:0.051224660128355026 acc:1.0
epoch:0 batch:184 loss:0.05647179111838341 acc:1.0
epoch:0 batch:185 loss:0.06343787163496017 acc:1.0
epoch:0 batch:186 loss:0.0622171126306057 acc:1.0
epoch:0 batch:187 loss:0.06592510640621185 acc:1.0
epoch:0 batch:188 loss:0.7560639977455139 acc:0.7833333333333333
epoch:0 batch:189 loss:3.0036473274230957 acc:0.0
epoch:0 batch:190 loss:2.9398398399353027 acc:0.0
epoch:0 ba

epoch:5 batch:115 loss:0.10218095034360886 acc:1.0
epoch:5 batch:116 loss:0.10080751776695251 acc:1.0
epoch:5 batch:117 loss:0.11387903243303299 acc:1.0
epoch:5 batch:118 loss:0.10301053524017334 acc:1.0
epoch:5 batch:119 loss:0.0975288450717926 acc:1.0
epoch:5 batch:120 loss:0.10469646751880646 acc:1.0
epoch:5 batch:121 loss:0.10468757152557373 acc:1.0
epoch:5 batch:122 loss:0.10795804113149643 acc:1.0
epoch:5 batch:123 loss:0.10126635432243347 acc:1.0
epoch:5 batch:124 loss:0.09755521267652512 acc:1.0
epoch:5 batch:125 loss:0.10160060226917267 acc:1.0
epoch:5 batch:126 loss:0.09682182222604752 acc:1.0
epoch:5 batch:127 loss:0.09428635239601135 acc:1.0
epoch:5 batch:128 loss:0.10974998772144318 acc:1.0
epoch:5 batch:129 loss:0.0941123366355896 acc:1.0
epoch:5 batch:130 loss:0.09276574105024338 acc:1.0
epoch:5 batch:131 loss:0.0891217589378357 acc:1.0
epoch:5 batch:132 loss:0.10002700984477997 acc:1.0
epoch:5 batch:133 loss:0.09383179992437363 acc:1.0
epoch:5 batch:134 loss:0.083509564

epoch:10 batch:50 loss:0.051596060395240784 acc:1.0
epoch:10 batch:51 loss:0.059436358511447906 acc:1.0
epoch:10 batch:52 loss:0.05812562629580498 acc:1.0
epoch:10 batch:53 loss:0.0547831654548645 acc:1.0
epoch:10 batch:54 loss:0.060103341937065125 acc:1.0
epoch:10 batch:55 loss:0.05221159756183624 acc:1.0
epoch:10 batch:56 loss:0.06009998545050621 acc:1.0
epoch:10 batch:57 loss:0.061195507645606995 acc:1.0
epoch:10 batch:58 loss:0.05315054580569267 acc:1.0
epoch:10 batch:59 loss:0.059993013739585876 acc:1.0
epoch:10 batch:60 loss:0.054048605263233185 acc:1.0
epoch:10 batch:61 loss:0.05431812256574631 acc:1.0
epoch:10 batch:62 loss:0.05402762070298195 acc:1.0
epoch:10 batch:63 loss:0.05628502368927002 acc:1.0
epoch:10 batch:64 loss:0.05472344160079956 acc:1.0
epoch:10 batch:65 loss:0.054340288043022156 acc:1.0
epoch:10 batch:66 loss:0.05365881323814392 acc:1.0
epoch:10 batch:67 loss:0.06349316984415054 acc:1.0
epoch:10 batch:68 loss:0.0606285035610199 acc:1.0
epoch:10 batch:69 loss:0.0

epoch:10 batch:214 loss:0.907828688621521 acc:0.0
epoch:10 batch:215 loss:0.926511824131012 acc:0.0
epoch:10 batch:216 loss:0.9015428423881531 acc:0.0
epoch:10 batch:217 loss:0.9005843997001648 acc:0.0
epoch:10 batch:218 loss:0.8878388404846191 acc:0.0
epoch:10 batch:219 loss:0.8716530799865723 acc:0.0
epoch:10 batch:220 loss:0.8439070582389832 acc:0.016666666666666666
epoch:10 batch:221 loss:0.8259273171424866 acc:0.0
epoch:10 batch:222 loss:0.8049310445785522 acc:0.0
epoch:10 batch:223 loss:0.7855069637298584 acc:0.03333333333333333
epoch:10 batch:224 loss:0.7651892900466919 acc:0.07272727272727272
epoch:15 batch:0 loss:0.044322334229946136 acc:1.0
epoch:15 batch:1 loss:0.04863972216844559 acc:1.0
epoch:15 batch:2 loss:0.04495922103524208 acc:1.0
epoch:15 batch:3 loss:0.04385264217853546 acc:1.0
epoch:15 batch:4 loss:0.04506655037403107 acc:1.0
epoch:15 batch:5 loss:0.048041291534900665 acc:1.0
epoch:15 batch:6 loss:0.048486676067113876 acc:1.0
epoch:15 batch:7 loss:0.049229782074689

epoch:15 batch:161 loss:0.033021457493305206 acc:1.0
epoch:15 batch:162 loss:0.02971765771508217 acc:1.0
epoch:15 batch:163 loss:0.031761884689331055 acc:1.0
epoch:15 batch:164 loss:0.2735968232154846 acc:0.95
epoch:15 batch:165 loss:3.616220712661743 acc:0.0
epoch:15 batch:166 loss:3.715786933898926 acc:0.0
epoch:15 batch:167 loss:3.527913808822632 acc:0.0
epoch:15 batch:168 loss:3.586193084716797 acc:0.0
epoch:15 batch:169 loss:1.6527693271636963 acc:0.5166666666666667
epoch:15 batch:170 loss:0.045383162796497345 acc:1.0
epoch:15 batch:171 loss:0.04848034307360649 acc:1.0
epoch:15 batch:172 loss:0.04754514992237091 acc:1.0
epoch:15 batch:173 loss:0.05756707116961479 acc:1.0
epoch:15 batch:174 loss:0.052135806530714035 acc:1.0
epoch:15 batch:175 loss:0.055414240807294846 acc:1.0
epoch:15 batch:176 loss:1.0166239738464355 acc:0.1
epoch:15 batch:177 loss:0.41874808073043823 acc:1.0
epoch:15 batch:178 loss:0.41345566511154175 acc:1.0
epoch:15 batch:179 loss:0.4452068507671356 acc:0.96666

In [27]:
model.eval()
x = torch.tensor( X_train[:,:,:]).float()
x = torch.where(torch.isnan(x), torch.full_like(x, 0), x)
    
y = y_train[:]
y = np.array(y)
y = torch.tensor(np.reshape(y,[-1,1]))
y = y.float()
    
x = x.cuda()
y = y.cuda()

x_valid = torch.tensor(X_valid[:,:,:]).float()
x_valid = torch.where(torch.isnan(x_valid), torch.full_like(x_valid, 0), x_valid)
    
y_val = y_valid[:]
y_val = np.array(y_val)
y_val = torch.tensor(np.reshape(y_val,[-1,1]))
y_val = y_val.float()

x_valid = x_valid.cuda()
y_val = y_val.cuda()

print("epoch:{} loss:{}".format(epoch,loss.item()))
print("train acc", evaluate_accuracy(x,y,model))
print("valid acc:",evaluate_accuracy(x_valid,y_val,model))

epoch:19 loss:0.18570905923843384
train acc 0.9332345313078918
valid acc: 0.951967978652435


In [28]:
import sklearn.metrics as sm
output,pre_out = model(x_valid)
output = torch.reshape(output,[-1,1])
output = output.ge(0.5)

output_cpu = output.cpu()
cr = sm.classification_report(y_valid[:,0], output_cpu)
print(cr)

              precision    recall  f1-score   support

           0       0.59      0.30      0.40        79
           1       0.96      0.99      0.97      1420

    accuracy                           0.95      1499
   macro avg       0.77      0.65      0.69      1499
weighted avg       0.94      0.95      0.94      1499

